<a href="https://colab.research.google.com/github/sanjib7777/PyTorch-Practice/blob/main/answer_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [ ]:
tokenize("my name is 'sanjib'?")

['my', 'name', 'is', 'sanjib']

In [ ]:
# vocab
vocab = {'<UNK>':0}

In [ ]:
# coonvert words to numerical
def build_vocab(row):

  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])
  merged_tokens = tokenized_question + tokenized_answer
  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)



In [ ]:
df.apply(build_vocab, axis = 1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [ ]:
len(vocab)

324

In [ ]:
def text_to_indices(text, vocab):
  index_text = []
  for token in tokenize(text):
    if token in vocab:
      index_text.append(vocab[token])
    else:
      index_text.append(vocab['<UNK>'])
  return index_text

In [ ]:
text_to_indices('where is france?', vocab)

[0, 2, 6]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df= df
    self.vocab = vocab

  def __len__(self):
    return df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)


In [ ]:
dataset = QADataset(df, vocab)

In [ ]:
dataset[2]

(tensor([10, 11, 12, 13, 14, 15]), tensor([16]))

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle = True)

In [ ]:
for question, answer in dataloader:
  print(question, answer)

tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([[36]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([[273]])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([[321]])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([[41]])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([[85]])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([[156]])
tensor([[ 10,  75, 111]]) tensor([[112]])
tensor([[ 10, 308,   3, 309, 310]]) tensor([[311]])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([[58]])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([[238]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([[173]])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([[54]])
tensor([[10, 75, 76]]) tensor([[77]])
tensor([[ 

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output

In [ ]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [ ]:
x = nn.Embedding(324, embedding_dim=50)
x

Embedding(324, 50)

In [ ]:
a = x(dataset[0][0])

In [ ]:
y = nn.RNN(50,64)

In [ ]:
# hidden output
y(a)[0].shape

torch.Size([6, 64])

In [ ]:
# final output
b = y(a)[1]

In [ ]:
z = nn.Linear(64,324)
z(b)

tensor([[ 0.2206,  0.1199,  0.1176,  0.2742, -0.4934, -0.3086,  0.3682, -0.2458,
          0.2144, -0.1338,  0.1742, -0.3140, -0.2362, -0.3623, -0.1207, -0.0114,
          0.3892, -0.1574,  0.3048, -0.0334,  0.4273,  0.2798,  0.0184,  0.2359,
          0.1839,  0.1003,  0.0912, -0.0579, -0.6157,  0.1659, -0.4843, -0.3027,
         -0.1018,  0.3278,  0.4064, -0.2334,  0.2717, -0.1742, -0.1226,  0.0889,
         -0.0431, -0.4879,  0.3243,  0.1511,  0.1428,  0.0662, -0.1427,  0.2433,
         -0.4097,  0.0524,  0.0997, -0.1361, -0.2847, -0.3751,  0.2091,  0.5865,
         -0.2129,  0.6985, -0.0732,  0.4154,  0.0672,  0.4390,  0.6439,  0.2149,
         -0.2236, -0.1299,  0.1914,  0.0212, -0.2873,  0.1051,  0.0186,  0.7598,
          0.0870,  0.0786,  0.3619, -0.2903, -0.3028, -0.3064,  0.5965, -0.2452,
         -0.2740,  0.3584,  0.0780, -0.1847,  0.3292,  0.2628,  0.2450,  0.3679,
         -0.2128, -0.0998,  0.3540, -0.1709, -0.0718, -0.4340,  0.3192, -0.0795,
          0.0270, -0.3175, -

In [ ]:
learning_rate = 0.001
epochs = 20

In [ ]:
model = SimpleRNN(len(vocab))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(epochs):
  total_loss = 0
  for question, answer in dataloader:
    optimizer.zero_grad()
    output = model(question)

    loss = criterion(output, answer[0])
    loss.backward()
    optimizer.step()
    total_loss = total_loss + loss.item()

  print(f'Epoch:{epoch+1}, Loss: {total_loss:4f}')


Epoch:1, Loss: 525.483435
Epoch:2, Loss: 446.891104
Epoch:3, Loss: 367.053582
Epoch:4, Loss: 312.413685
Epoch:5, Loss: 263.536434
Epoch:6, Loss: 216.880839
Epoch:7, Loss: 174.769442
Epoch:8, Loss: 138.297021
Epoch:9, Loss: 107.656301
Epoch:10, Loss: 83.540725
Epoch:11, Loss: 65.222471
Epoch:12, Loss: 51.303645
Epoch:13, Loss: 41.127218
Epoch:14, Loss: 33.318967
Epoch:15, Loss: 27.248827
Epoch:16, Loss: 22.623102
Epoch:17, Loss: 18.975507
Epoch:18, Loss: 16.204832
Epoch:19, Loss: 13.815862
Epoch:20, Loss: 11.991224


In [ ]:
def predict(model, question, threshold=0.5):
  numerical_question = text_to_indices(question, vocab)
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  output = model(question_tensor)
  print(output.shape)
  probs = torch.nn.functional.softmax(output, dim =1)
  value, index = torch.max(probs, dim=1)
  if value< threshold:
    print("i don't know")
  else:
    print(list(vocab.keys())[index])


In [ ]:
predict(model, 'What is the capital of Germany?	')

torch.Size([1, 324])
berlin
